In [1]:
import numpy as np
import torch

from dataset import Mnist
from model import ClassificationMnist

from random import choice

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import top_k_accuracy_score

from collections import Counter



In [2]:
model = ClassificationMnist()

In [3]:
train_data = Mnist("train-images.idx3-ubyte", "train-labels.idx1-ubyte")
test_data = Mnist("t10k-images.idx3-ubyte", "t10k-labels.idx1-ubyte")

train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=256, shuffle=True, num_workers=12
)

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=256, num_workers=12
)

In [4]:
for i in train_loader:
    break

In [5]:
model(i[0].to(torch.float32)).shape

torch.Size([256, 10])

In [6]:
128 * 7 * 7

6272

In [7]:
model = torch.load('ae250.model').encoder.cuda()
class_model = torch.load('classification310.model').cuda()

model.eval()
class_model.eval()

ClassificationModel(
  (conv_block): Sequential(
    (0): Conv2d(3, 8, kernel_size=(5, 5), stride=(3, 3))
    (1): Dropout2d(p=0.3, inplace=False)
    (2): ReLU()
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
  )
  (fc_block): Sequential(
    (0): Linear(in_features=1600, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout2d(p=0.2, inplace=False)
    (3): Linear(in_features=1024, out_features=100, bias=True)
  )
  (final): LogSoftmax(dim=1)
)

In [8]:
test_data = Cifar100("CIFAR100/TEST")

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=len(test_data), num_workers=12
)

NameError: name 'Cifar100' is not defined

In [ ]:
data, classes = next(iter(test_loader))

In [ ]:
data = data.cuda()
data = data.to(torch.float32)
data /= 255
classes = classes.numpy()

top 3, top 5 and top 10 score

In [ ]:
predict = class_model(model(data)).detach().cpu().numpy()
top_k_accuracy_score(classes, predict, k=3), top_k_accuracy_score(classes, predict, k=5), top_k_accuracy_score(classes, predict, k=10)

In [ ]:
error_per_class = Counter()

for i in range(len(test_data.classes)):
    error_per_class[i] += sum((predict.argmax(1) == classes)[classes == i]).item()


In [ ]:
count_classes = 10
each_classes_count = 100 
displayed_classes = [test_data.classes[x[0]] for x in error_per_class.most_common()[:count_classes]]
displayed_classes

In [ ]:
id_disp_classes = [test_data.classes.index(x) for x in displayed_classes]

new_data = data[classes == id_disp_classes[0]][:each_classes_count]

for i in id_disp_classes[1:]:
    new_data = torch.cat((new_data, data[classes == i][:each_classes_count]))

data = new_data
data.shape

In [ ]:
data = model(data)

In [ ]:
data = class_model.conv_block(data)
data = data.reshape(-1, 1600)
data = class_model.fc_block(data)
data.shape

In [ ]:
data = data.detach().cpu().numpy()

In [ ]:
data.shape

In [ ]:
pca = PCA(n_components=2)
pca.fit_transform(data)
x = pca.transform(data)

two-dimensional representation of elements

In [ ]:
fig = plt.figure(figsize = (15,8))
ax = fig.add_subplot() 

for i, id in enumerate(id_disp_classes):
    j = x[i * each_classes_count: (i + 1) * each_classes_count]
    ax.scatter(j[:, 0], j[:, 1], s=10)

ax.legend(displayed_classes)

ax.grid()